In [18]:
import nltk
import matplotlib
%matplotlib inline
import pandas as pd

In [19]:
from urllib.request import urlopen
my_url = "http://gutenberg.net.au/ebooks02/0200331.txt"
file = urlopen(my_url)
raw = file.read()
orlando = raw.decode()
o_tokens = nltk.word_tokenize(orlando)

In [20]:
o_text = o_tokens[872:-5]

In [21]:
orl = nltk.Text(o_text)
orl

<Text: He -- for there could be no doubt...>